In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import entropy
import datetime
import copy
import time
import wandb
import pickle as pkl
import sys
sys.path.append('../../')
from data.processing import get_data
import models
from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new
from main.seir.sensitivity import calculate_sensitivity_and_plot
from utils.generic.create_report import save_dict_and_create_report
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles
from viz.fit import plot_histogram, plot_all_histograms, plot_mean_variance, plot_scatter, plot_kl_divergence, plot_heatmap_distribution_sigmas, plot_all_params, plot_all_losses, plot_all_buckets, plot_cv_in_params, plot_recovery_loss, plot_confidence_interval
import yaml
from data.dataloader import SimulatedDataLoader

In [ ]:
simulated_config_filename = 'seirhd_fixed.yaml'
with open(os.path.join("../../configs/simulated_data/", simulated_config_filename)) as configfile:
    simulated_config = yaml.load(configfile, Loader=yaml.SafeLoader)    
actual_params = simulated_config['params']

In [ ]:
predictions_dict = {}

In [ ]:
config_filename = 'default.yaml'
config = read_config(config_filename)

In [ ]:
# varying_perc = np.array([-0.1,-0.25,-0.4,-0.5,-0.6,-0.75,-0.9])
varying_perc = np.array([-0.5])
varying_perc = np.concatenate([np.flipud(varying_perc),[0],-1*varying_perc])
varying_perc   

In [ ]:
n_iters = 2
required_params = actual_params.copy()
del required_params['N']
losses = {}

for param,val in required_params.items():
    losses[param] = {}
    print(param)
    for perc_change in varying_perc : 
        losses[param][perc_change] = []
        test_value = val + val*perc_change
        for i in range(n_iters):
            config_params = copy.deepcopy(config['fitting'])
            if param in config_params['variable_param_ranges']:
                del config_params['variable_param_ranges'][param] 
            config_params['default_params'][param] = test_value
            config_params['fitting_method_params']['num_evals'] = 250
            predictions_dict = single_fitting_cycle(**config_params)
            losses[param][perc_change].append(predictions_dict['df_loss'])
    break

In [ ]:
agg_losses = {}
for param in losses:
    agg_losses[param] = {}
    for perc_change in losses[param]:
        agg_losses[param][perc_change] = 0
        for i in range(len(losses[param][perc_change])):
            df = losses[param][perc_change][i]
            agg_losses[param][perc_change] += df['train'].mean()
        agg_losses[param][perc_change] / len(losses[param][perc_change])
print(agg_losses)

In [ ]:
for param in agg_losses:
    plt.plot(agg_losses[param].keys(),agg_losses[param].values())